<a href="https://colab.research.google.com/github/GeorgeFane/MDining-Scraper/blob/master/MDining_Scraper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Press Ctrl+F9 or Runtime > Run all and Scroll Down

In [ ]:
%load_ext google.colab.data_table
import pandas as pd
import numpy as np
 
import requests
from lxml.html import fromstring, tostring
 
from datetime import datetime as dt
from pytz import timezone

In [ ]:
# page targets
halls = [
    'Bursley',
    'East Quad',
    'Markley',
    'Mosher-Jordan',
    'North Quad',
    'South Quad',
    'Twigs at Oxford',
    'MDining To-Go/Kosher Kitchen',
    'MDining To-Go/Central Campus Recreation Building',
    'MDining To-Go/Michigan Union',
    'MDining To-Go/South Quad',
] 

In [ ]:
out = lambda tree: print(tostring(tree).decode('utf-8'))
 
# scrape from page HTML
getTitles = lambda: tree.xpath('//span[@class="calhours-title"]/text()')
 
getTimes = lambda: [time.replace('\xa0', ' ') for time in tree.xpath('//span[@class="calhours-times"]/text()')]
 
getCourses = lambda: [div.xpath('.//div[@class="item-name"]/text()') for div in tree.xpath('//div[@class="courses"]')]
 
tz = timezone("America/Detroit")
ptime = lambda string, form: dt.strptime(string, form).time()
strnow = lambda: dt.strftime(dt.now(tz), '%B %d, %Y %I:%M %P')
 
def isOpen(time):
    start, end = time.split(' ‑ ')
    return ptime(start, '%I:%M %p') < dt.now(tz).time() < ptime(end, '%I:%M %p')

In [ ]:
tree = 0
def execute(hall):
    link = 'https://dining.umich.edu/menus-locations/dining-halls/' + hall.replace(' ', '-')
    r = requests.get(link)
    global tree
    tree = fromstring(r.content)

    labels = [text.strip() for text in set(tree.xpath('//a[@href="#"]/text()'))]
    top = [(title, time) for title, time in zip(getTitles(), getTimes()) if title in labels]
    return [(isOpen(time), hall, title, time, course) for (title, time), course in zip(top, getCourses())]

In [ ]:
headers = 'isOpen Hall Meal Time Foods'.split()
table = [x for hall in halls for x in execute(hall)]
df = pd.DataFrame(dict(zip(headers, np.array(table).T)))
 
# display
print('Updated ' + strnow()) 
df

Updated October 30, 2020 03:04 pm


,isOpen,Hall,Meal,Time,Foods
0,False,Bursley,Breakfast,7:00 am ‑ 10:30 am,"[Oatmeal , Brazilian Kale , Vegan Pancakes , C..."
1,True,Bursley,Lunch,10:30 am ‑ 4:30 pm,"[Manhattan Clam Chowder , Shrimp Po Boy, Saute..."
2,False,Bursley,Dinner,4:30 pm ‑ 8:00 pm,"[Manhattan Clam Chowder , Roasted Zucchini , R..."
3,False,East Quad,Breakfast,7:00 am ‑ 10:30 am,"[Cream of Wheat , Scrambled Eggs , Scrambled ..."
4,True,East Quad,Lunch,10:30 am ‑ 4:30 pm,"[Authentic Minestrone , Chocolate Chunk Cookie..."
5,False,East Quad,Dinner,4:30 pm ‑ 8:00 pm,"[Manhattan Clam Chowder , Garden Salad, Chocol..."
6,False,Markley,Breakfast,7:00 am ‑ 10:30 am,"[Organic Steel-Cut Oatmeal, Scrambled Eggs , ..."
7,False,Markley,Lunch,10:30 am ‑ 2:00 pm,"[Black Bean Soup , Manhattan Clam Chowder , ..."
8,False,Markley,Dinner,4:30 pm ‑ 8:00 pm,"[Black Bean Soup , Manhattan Clam Chowder , ..."
9,False,Mosher-Jordan,Breakfast,7:00 am ‑ 10:30 am,"[Oatmeal , Fresh Fruit , Hard Cooked Eggs, Pot..."
